In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entitySourceQuery", "")
dbutils.widgets.text("entityColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save the parameter to a variable:
entityName = dbutils.widgets.get('entityName')
connection = json.loads(dbutils.widgets.get('entitySourceQuery'))
sourceSystemName = dbutils.widgets.get('sourceSystem')

# Define query:
query = connection['sourceQuery']

In [0]:
# Check source system:
if sourceSystemName == 'TotesysDB':
  
  # If entity is date, do not ingest:
  if entityName == 'date':
    dbutils.notebook.exit('None')
  
  else:
    # Create DataFrame from SQL Server:
    df = (spark.read
      .format("sqlserver")
      .option("host", "framework-j1-sv.database.windows.net")
      .option("port", "1433")
      .option("user", "adminsjp")
      .option("password", db_password)
      .option("database", "totesys")
      .option("query", query)
      .load()
    )

In [0]:
# Set folder path to todays date:
folder_path = datetime.datetime.now().strftime("%Y/%m/%d")

# Check data exists:
if df.count() > 0:
    # Set location for parquet:
    location = BRONZE + f"/{sourceSystemName}/{entityName}/{folder_path}/"
    
    # Write parquet to data lake:
    df.write.mode('overwrite').format('parquet').option('header', 'true').save(location)


In [0]:
dbutils.notebook.exit(location)